In [1]:
import numpy as np
import glob
import random
from segment2d import *
import nibabel as nib
from ipywidgets import interact
import skimage
import matplotlib.pyplot as plt

# import albumentations as A
from torchvision import transforms
import os

import csv
import kornia as K
from tqdm import tqdm
from torch.utils.data import DataLoader
import yaml
import pandas as pd
# natsort
from natsort import natsorted
import shutil
# import SimpleITK as sitk
from segment2d import *
from skimage.transform import resize
def compute_iou(mask1, mask2, class_id, eps=1e-6):
    mask1_binary = mask1==class_id
    mask2_binary = mask2==class_id
    intersection = np.sum(mask1_binary*mask2_binary)
    union = np.sum(mask1_binary)+np.sum(mask2_binary)-intersection
    return (intersection+eps)/(union+eps)


# test emidec

In [36]:
list_image_normal = natsorted(glob.glob("./emidec-dataset-1.0.1/Case_N*/I*/*"))
list_image_pathologic = natsorted(glob.glob("./emidec-dataset-1.0.1/Case_P*/I*/*"))
# split the dataset into training and validation and test with sample ratio 70:10:20
random.seed(42)
random.shuffle(list_image_normal)
random.shuffle(list_image_pathologic)
train_normal = list_image_normal[:int(0.7 * len(list_image_normal))+1]
val_normal = list_image_normal[int(0.7 * len(list_image_normal))+1 : int(0.8 * len(list_image_normal))+1]
test_normal = list_image_normal[int(0.8 * len(list_image_normal))+1:]
train_pathologic = list_image_pathologic[:int(0.7 * len(list_image_pathologic))]
val_pathologic = list_image_pathologic[int(0.7 * len(list_image_pathologic)) : int(0.8 * len(list_image_pathologic))]
test_pathologic = list_image_pathologic[int(0.8 * len(list_image_pathologic)) :]

list_train = train_normal + train_pathologic
list_val = val_normal + val_pathologic
list_test = test_normal + test_pathologic



In [37]:
data = np.load("emidec_train_full/Case_N006_1.npz")
image = data["image"]
mask = data["mask"]

In [38]:
dict_value = {}
count = 0
for image_path in list_train+list_val+list_test:
    image = nib.load(image_path).get_fdata()
    mask = nib.load(image_path.replace("Images", "Contours")).get_fdata()
    resize_image, restore_info = crop_resize_image(image, new_dim=256)
    resize_mask = crop_resize_mask(mask, restore_info)
    inverted_mask = restore_mask(resize_mask, restore_info)
    if np.linalg.norm(mask-inverted_mask) > 1:
        # print(image_path, np.linalg.norm(mask-inverted_mask))
        count += 1
print(count)

    # # calculate the number of pixel = 0, 1,2,3,4 in the mask
    # unique, counts = np.unique(padded_mask, return_counts=True)
    # for i in range(len(unique)):
    #     if unique[i] in dict_value:
    #         dict_value[unique[i]] += counts[i]
    #     else:
    #         dict_value[unique[i]] = counts[i]

99


In [39]:
from kornia.augmentation import *

aug = AugmentationSequential(
    RandomHorizontalFlip(p=0.5),
    RandomVerticalFlip(p=0.5),
    RandomResizedCrop((128, 128), scale=(0.8, 1.25), ratio=(0.8, 1.25), p=0.5),
    data_keys=["input", "mask"],
)

In [40]:
# visualize the image and mask in z axis using interact, image and mask are in one slice
def plot_image_mask_z(image, mask, padded_image, padded_mask, z):
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(image[..., z], cmap="gray")
    ax[0].set_title("Image uncut")
    ax[0].imshow(mask[..., z], cmap="jet", alpha=0.3)
    ax[1].imshow(padded_image[..., z], cmap="gray")
    ax[1].set_title("Image cut")
    ax[1].imshow(padded_mask[..., z], cmap="jet", alpha=0.3)

    plt.show()

interact(lambda z: plot_image_mask_z(image, mask, resize_image, resize_mask, z), z=(0, image.shape[-1] - 1))

interactive(children=(IntSlider(value=2, description='z', max=5), Output()), _dom_classes=('widget-interact',)…

<function __main__.<lambda>(z)>

# test acdc


In [3]:
# gt : ACDC/database/training/patient001/patient001_frame01_gt.nii.gz
from numpy import pad


list_patient_train = natsorted(glob.glob("ACDC/database/training/patient*"))
list_patient_test = natsorted(glob.glob("ACDC/database/testing/patient*"))
for patient_path in list_patient_train+list_patient_test:
    # load the image and gt with nib
    # print(patient_path)
    for gt_img_path in glob.glob(patient_path + "/*gt.nii.gz"):
        img_path = gt_img_path.replace("_gt", "")
        image = nib.load(img_path).get_fdata()
        mask = nib.load(gt_img_path).get_fdata()
        new_image, restore_info = crop_resize_image(image, 256)
        new_mask = crop_resize_mask(mask, restore_info)
        inverted_mask = restore_mask(new_mask, restore_info)
        for class_index in range(1, 4):
            if compute_iou(mask, inverted_mask, class_index) < 0.9:
                print(gt_img_path, compute_iou(mask, inverted_mask, class_index))
    

In [175]:
np.sum(mask[...,1]==1), np.sum(inverted_mask[...,1]==1)

(1706, 1706)

In [176]:
np.sum(mask[...,-3]), np.sum(inverted_mask[...,-3])

(3131, 3131)

In [177]:
new_image, restore_info = crop_resize_image(image)
new_mask = crop_resize_mask(mask, restore_info)
inverted_mask = restore_mask(new_mask, restore_info)

In [13]:
data = np.load("ACDC_train/patient001_4.npz")
image = data["image"]
mask = data["mask"]
print(image.shape)
print(mask.shape)


(256, 256, 1)
(256, 256)


In [16]:
    with open("csv_files/ACDC_val.csv", mode="r") as f:
        reader = csv.DictReader(f)
        list_val_subject = [row["path"] for row in reader]

In [18]:
val_dataset = Test_Volume_Loader(list_val_subject)
batch = val_dataset[0]
mask = batch["mask"]

In [19]:
mask.shape

(256, 216, 10)

In [25]:
image = batch["image"].numpy()[:, 0].transpose(1, 2, 0)
new_image = image.copy()
resize_mask = crop_resize_mask(mask, batch["resize_info"])
new_mask = restore_mask(resize_mask, batch["resize_info"])

In [26]:
image.shape

(256, 256, 10)

In [ ]:
def plot_image_mask_z(image, mask, padded_image, padded_mask, z):
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(image[..., z], cmap="gray")
    ax[0].set_title("Image uncut")
    ax[0].imshow(mask[..., z], cmap="jet", alpha=0.3)
    ax[1].imshow(padded_image[..., z], cmap="gray")
    ax[1].set_title("Image cut")
    ax[1].imshow(padded_mask[..., z], cmap="jet", alpha=0.3)
    # off the axis
    ax[0].axis('off')
    ax[1].axis('off')


interact(lambda z: plot_image_mask_z(image, mask, new_image, new_mask, z), z=(0, image.shape[-1] - 1))
interact(lambda z: plot_image_mask_z(image, mask, image, inverted_mask, z), z=(0, image.shape[-1] - 1))

interactive(children=(IntSlider(value=4, description='z', max=9), Output()), _dom_classes=('widget-interact',)…

interactive(children=(IntSlider(value=4, description='z', max=9), Output()), _dom_classes=('widget-interact',)…

<function __main__.<lambda>(z)>